# Setup

In [137]:
import pandas as pd

In [138]:
prisons = pd.read_csv('../daymet_air_temp/Input/prisons_clean_03042024.csv')

# Analysis

In [139]:
prisons.columns

Index(['FACILITYID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4', 'TYPE',
       'STATUS', 'POPULATION', 'COUNTY', 'COUNTRY', 'SECURELVL'],
      dtype='object')

In [140]:
# Count the number of instances where the capacity is less than 1
capacity_error = len(prisons[prisons['POPULATION'] < 1])

total_rows = len(prisons)

# Display the count
print(f'The number of instances where Population < 1 (no data): {capacity_error} out of {total_rows}')

The number of instances where Population < 1 (no data): 116 out of 1614


In [141]:
sorted_prisons = prisons.sort_values(by=['STATE', 'POPULATION'], ascending=[True, False])

In [142]:
prisons_filtered = prisons[prisons['STATE'] != 'DC']
top_3_per_state = prisons_filtered.groupby('STATE').head(3)

In [143]:
highpop_prisons = top_3_per_state.copy(deep=True)

In [144]:
highpop_prisons.shape

(150, 13)

In [145]:
ac = pd.read_csv('../ac_datasets/ac_status.csv')

In [146]:
ac.head()

,facility_id,prison_name,state,ac_status,ac_status_details,address,building_completion_year,notes
0,10002400,Bridgeport Correctional Center,CT,Yes,Partial,"1106 North Avenue Bridgeport, CT 06604",1958,"1958 listed as the open date, assume it is the..."
1,10002399,Brooklyn Correctional Institution,CT,Yes,Full,"59 Hartford Road Brooklyn, CT 06234",1820,"Additions added in 1842, 1863, 1990"
2,10000937,Cheshire Correctional Institution,CT,Yes,Full,"900 Highland Avenue Cheshire, CT 06410",1913,Addition in 1956
3,10000931,Corrigan Correctional Center,CT,Yes,Full,"986 Norwich-New London Turnpike Uncasville, CT...",1957,"Merged with buildings opened in 1994, 1991"
4,10006149,Cybulski Community Reintegration Center,CT,Yes,Full,"264 Bilton Road Somers, CT 06071",1990,Merged with a building opened in 1993


In [147]:
highpop_prisons['FACILITYID'] = highpop_prisons['FACILITYID'].astype(str)
ac['facility_id'] = ac['facility_id'].astype(str)


In [148]:
duplicates_in_ac = ac[ac.duplicated(subset='facility_id', keep=False)]
duplicates_in_highpop_prisons = highpop_prisons[highpop_prisons.duplicated(subset='FACILITYID', keep=False)]

In [149]:
duplicates_in_ac

,facility_id,prison_name,state,ac_status,ac_status_details,address,building_completion_year,notes
22,10000298,GA Diagnostic Class Prison,GA,Yes,Partial,"2978 Hwy 36 West Prison Blvd, Jackson, GA 30233",1968,NaN
23,10000298,Hancock State Prison,GA,Yes,Partial,"701 Prison Blvd, Sparta, GA 31087",1990,NaN
313,nan,Beaumont Correctional Center,VA,Yes,Full,"3500 Beaumont Rd., Beaumont, VA 23014",not provided,NaN
315,10001597,Brunswick CCAP,VA,Yes,Full,"1147 PLANTERS RD, LAWRENCEVILLE, VA 23868",not provided,facility not on website but provided to us in ...
335,10001597,Lawrenceville Correctional Center,VA,No,NaN,"1607 Planters Road, Lawrenceville, VA 23868",not provided,NaN
346,nan,State Farm Enterprise Unit,VA,No,NaN,"3600 Woods Way, State Farm, VA 23160",not provided,NaN
347,nan,State Farm Work Center,VA,No,NaN,"1954 State Farm Road, State Farm, VA 23160",not provided,NaN
371,10000053,Wyoming Medium Correctional Institution (WMCI),WY,Yes,Full,"7076 Road 55F Torrington, WY 82240",2009,"For the Admin, Programing & Housing Building"
374,10000053,WHCC Wyoming Honor Conservation Camp,WY,Yes,Full,"40 Pippin Road Newcastle, WY 82701",1999,Boot Camp Building (Barracks) was built in 200...


In [150]:
prison_builds = pd.merge(highpop_prisons, ac, left_on='FACILITYID', right_on='facility_id', how='left')

prison_builds.fillna('N/A', inplace=True)

In [151]:
prison_builds.shape

(151, 21)

In [152]:
def extract_first_year(year_span):
    if '-' in year_span:
        return year_span.split('-')[0]
    return year_span

In [153]:
prison_builds['building_completion_year'] = prison_builds['building_completion_year'].apply(extract_first_year)

In [154]:
prison_builds.shape

(151, 21)

In [157]:
prison_builds = prison_builds.sort_values(by='state')

In [158]:
prison_builds.to_csv('prison_builds_study.csv')